# BoTorch Tutorials

The following information is summarised from the main BoTorch website.
https://botorch.org/docs/introduction.html

I have decided to summarise it to extract the core information from the documents to get started.


## Introduction to BoTorch

BoTorch is a library for Bayesian Optimisation research built on top of PyTorch. 

Bayesian Optimisation (BayesOpt) is an established technique for sequential optimisation of costly-to-evaluate black box functions. It can be applied to a wide variety of problems, including hyperparameter optimisation for machine learning algorithms, A/B testing, as well as many scientific and engineering problems.

## Why Botorch?

1. Improved Developer Efficiency
Modular and easily extensible interface for composing 

*   Bayesian Optimisation primitives (include probabilitic mdels, acquisition functions and optimisers)
*   Utilise quasi-Monte-Carlo acquisition functions


2. State-of-the-art Modelling
*   Provide support for state-of-the-art probabilisitc models in GPyTorch (Library for efficient, scalable Gaussian Process implemented in PyTorch).
*   Features include multi-task GPs, deep kernel learning, deep GPs, and approximate inference


3. Harnessing the features of PyTorch
*   Auto-differentiation, GPU implementations and dynamic computation graph 

# Simple Programme to get started

In [1]:
# ===============
# Install BoTorch
# ===============

# Via conda
# conda install botorch -c pytorch -c gpytorch

# Via pip
!pip install botorch

     |████████████████████████████████| 225kB 8.4MB/s 
     |████████████████████████████████| 245kB 14.4MB/s 
  Created wheel for gpytorch: filename=gpytorch-1.1.1-py2.py3-none-any.whl size=400467 sha256=69c31a85387f48ab60cd0be642d3a40b7eef8ed5f2eb0b505c0825f9d31fdc9d
  Stored in directory: /root/.cache/pip/wheels/e5/a5/29/4dafc0624adf678108e0067836556f0c72588e85d851d78ae0
Successfully built gpytorch


In [8]:
# ===============
# Fitting a Model
# ===============

import torch
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_model
from botorch.utils import standardize
from gpytorch.mlls import ExactMarginalLogLikelihood


# Create training set
train_X = torch.rand(10,2)  # Create random matrix 10 rows x 2 columns
Y = 1-torch.norm(train_X,dim=-1,keepdim=True) # Generate Y = 1-sqrt(X1^2+X2^2)

# randn_like(.)
# Returns a tensor with the same size as input that is filled with 
# random numbers from a normal distribution with mean 0 and variance 1
Y = Y + 0.1*torch.randn_like(Y)  # Add some noise (Size of Y)

# standardize(.)
# Standardizes (zero mean, unit variance) a tensor by dim=-2.
# If the tensor is single-dimensional, simply standardizes the tensor. 
# If for some batch index all elements are equal (of if there is only a single 
# data point), this function will return 0 for that batch index.
train_Y = standardize(Y) # Standardisation of Y for training

# Gaussian Process Regression models based on GPyTorch models.
# SingleTaskGP(train_X, train_Y, likelihood=None, covar_module=None, outcome_transform=None)
# Single-task exact GP model
# A single-task exact GP using relatively strong priors on the Kernel hyperparameters, 
# which work best when covariates are normalized to the unit cube and outcomes are standardized 
# (zero mean, unit variance).
# This model works in batch mode (each batch having its own hyperparameters). When the 
# training observations include multiple outputs, this model will use batching to model outputs 
# independently.
# Use this model when you have independent output(s) and all outputs use the same training data. 
# If outputs are independent and outputs have different training data, use the ModelListGP. 
# When modeling correlations between outputs, use the MultiTaskGP.
# Parameters
# train_X (Tensor) – A batch_shape x n x d tensor of training features.
# train_Y (Tensor) – A batch_shape x n x m tensor of training observations.
# likelihood (Optional[Likelihood]) – A likelihood. 
#       If omitted, use a standard GaussianLikelihood with inferred noise level.
# covar_module (Optional[Module]) – The module computing the covariance (Kernel) matrix. 
#       If omitted, use a MaternKernel.
# outcome_transform (Optional[OutcomeTransform]) – An outcome transform that is applied 
#       to the training data during instantiation and to the posterior during inference 
#       (that is, the Posterior obtained by calling .posterior on the model will be on 
#       the original scale).
gp = SingleTaskGP(train_X,train_Y)

# These are modules to compute (or approximate/bound) the marginal log likelihood (MLL) 
# of the GP model when applied to data.
# These models are typically used as the “loss” functions for GP models
# The exact marginal log likelihood (MLL) for an exact Gaussian process with a Gaussian likelihood.
mll = ExactMarginalLogLikelihood(gp.likelihood,gp)


# fit_gpytorch_model(mll, optimizer=<function fit_gpytorch_scipy>, **kwargs)
# Fit hyperparameters of a GPyTorch model.
# On optimizer failures, a new initial condition is sampled from the hyperparameter 
# priors and optimization is retried. The maximum number of retries can be passed 
# in as a max_retries kwarg (default is 5).
# Returns MarginalLogLikelihood with optimized parameters. 
# For GP models, we are picking the best kernel for the GP.
fit_gpytorch_model(mll)

ExactMarginalLogLikelihood(
  (likelihood): GaussianLikelihood(
    (noise_covar): HomoskedasticNoise(
      (noise_prior): GammaPrior()
      (raw_noise_constraint): GreaterThan(1.000E-04)
    )
  )
  (model): SingleTaskGP(
    (likelihood): GaussianLikelihood(
      (noise_covar): HomoskedasticNoise(
        (noise_prior): GammaPrior()
        (raw_noise_constraint): GreaterThan(1.000E-04)
      )
    )
    (mean_module): ConstantMean()
    (covar_module): ScaleKernel(
      (base_kernel): MaternKernel(
        (lengthscale_prior): GammaPrior()
        (raw_lengthscale_constraint): Positive()
        (distance_module): Distance()
      )
      (outputscale_prior): GammaPrior()
      (raw_outputscale_constraint): Positive()
    )
  )
)

In [0]:
# ==================================
# Constructing acquisition function
# ==================================

from botorch.acquisition import UpperConfidenceBound
UCB = UpperConfidenceBound(gp,beta=0.1)


In [7]:
# =================================
# Optimise the acquisition function
# =================================

from botorch.optim import optimize_acqf

bounds = torch.stack([torch.zeros(2),torch.ones(2)])


# optimize_acqf(acq_function, bounds, q, num_restarts, raw_samples, options=None, 
#               inequality_constraints=None, equality_constraints=None, fixed_features=None, 
#               post_processing_func=None, batch_initial_conditions=None, return_best_only=True, 
#               sequential=False)
# 
# Generate a set of candidates via multi-start optimization.
#
# ==========
# Parameters
# ==========
#
# acq_function (AcquisitionFunction) : An Acquisition Function.
# bounds (Tensor) : A 2 x d tensor of lower and upper bounds for each column of X.
# q (int) : The number of candidates.
# num_restarts (int) : The number of starting points for multistart acquisition function optimization.
# raw_samples (int) : The number of samples for initialization.
# options (Optional[Dict[str, Union[bool, float, int, str]]]) : Options for candidate generation.
# constraints (equality) : A list of tuples (indices, coefficients, rhs), with each tuple 
#                          encoding an inequality constraint of the form sum_i 
#                          (X[indices[i]] * coefficients[i]) >= rhs
# constraints : A list of tuples (indices, coefficients, rhs), with each tuple encoding an 
#               inequality constraint of the form sum_i (X[indices[i]] * coefficients[i]) = rhs
# fixed_features (Optional[Dict[int, float]]) : A map {feature_index: value} for features that should 
#                                               be fixed to a particular value during generation.
# post_processing_func (Optional[Callable[[Tensor], Tensor]]) : A function that post-processes an 
#                                                               optimization result appropriately 
#                                                  (i.e., according to round-trip transformations).
# batch_initial_conditions (Optional[Tensor]) : A tensor to specify the initial conditions. 
#                                               Set this if you do not want to use default 
#                                               initialization strategy.
# return_best_only (bool) : If False, outputs the solutions corresponding to all random restart 
#                           initializations of the optimization.
# sequential (bool) : If False, uses joint optimization, otherwise uses sequential optimization.
#
# =======
# Returns
# =======
# 1. (num_restarts) x q x d-dim tensor of generated candidates
# 2. tensor of associated acquisiton values
#    If sequential=False, this is a (num_restarts)-dim tensor of joint acquisition values 
#    (with explicit restart dimension if return_best_only=False).
#    If sequential=True, this is a q-dim tensor of expected acquisition values conditional 
#    on having observed canidates 0,1,…,i-1.
candidate,acq_value = optimize_acqf(UCB,bounds=bounds,q=1,num_restarts=5,raw_samples=20,)
candidate

tensor([[0.2485, 0.2005]])

## Constrained BO with qEI and qNEI

In this section, we will use the batch Expected Improvement (qEI) and batch Noist Expected Improvement (qNEI) acquisition functions to optimize a constrained version of the synthetic Hartmann test function.





In [0]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.double